In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df=pd.read_csv('Nat_Gas.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Dates']=pd.to_datetime(df['Dates'])

In [ ]:
import plotly.express as px


In [ ]:
fig=px.line(df,x='Dates',y='Prices',title='Natural Gas Prices')
fig.show()

i can see that there is a seasonality where the prices go up in december, january, feb and march and it starts going down where there is also a repetitive behavior where it goes to the minimum in may, jun, july and august and then starts to go up again

i also notice an upward trend on the whole timeline where the prices are actually going up within the four years. 


# Natural Gas purchase price estimation

1- set date as index for time series interpolation 



In [ ]:
df.set_index('Dates',inplace=True)


2- Interpolating the price data for any missing date between the given monthly prices 


In [ ]:
df_interpolated=df.resample('D').interpolate('linear')

In [ ]:
df_interpolated

# Extrapolate future prices using a simple time series model 

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

#train a simple time series model to predict future prices 
model=ExponentialSmoothing(df_interpolated['Prices'], trend='add', seasonal='add',seasonal_periods=365)
model_fit=model.fit()

#Generate future dates for the next year (12 months)
future_dates=pd.date_range(start=df_interpolated.index[-1],periods=365,freq='d')

#Forecast future prices for 12 months
future_prices=model_fit.forecast(steps=365)

#create a dataframe for future prices
future_df=pd.DataFrame({'Dates':future_dates, 'Prices':future_prices})
future_df.set_index('Dates',inplace=True)

future_df

In [ ]:
def estimate_gas_price(input_date):
    
    #check if date is in the past or present
    if input_date in df_interpolated.index:
        return df_interpolated.loc[input_date]['Prices']
    
    #check if the date is in the future (within the extrapolated range )
    elif input_date in future_df.index:
        return future_df.loc[input_date]['Prices']
    else:
        return "Date is out of range for estimation"
    

In [ ]:
input_date='2024-10-30'
price_estimate=estimate_gas_price(input_date)
print(f"Estimated price on {input_date}: {price_estimate}")